In [1]:
import numpy as np
from scipy import sparse
import pandas as pd
import torch
import argparse
import json
import os

In [6]:

with open("../paths.json", "r") as f:

        paths = json.load(f)
        csv = paths['eICU_path'] 
        graph = paths['graph_dir']